In [1]:
%%capture
!git clone https://github.com/ynop/py-ctc-decode
!pip3 install transformers datasets torchaudio jiwer psutil
!pip3 install https://github.com/kpu/kenlm/archive/master.zip
!wget http://hetzner.depia.wiki/idwiki.arpa

In [2]:
%cd py-ctc-decode

/home/user/py-ctc-decode


In [13]:
vocab = ['m',
 'h',
 'r',
 'e',
 'z',
 'a',
 ' ',
 'k',
 'g',
 's',
 'x',
 'l',
 'f',
 'b',
 'v',
 'n',
 'c',
 'd',
 'p',
 'o',
 't',
 'y',
 'w',
 'i',
 'j',
 'u',
 '',
 '']

In [17]:
import ctcdecode
import numpy as np

vocabulary = vocab
alpha = 2.5 # LM Weight
beta = 0.0 # LM Usage Reward
word_lm_scorer = ctcdecode.WordKenLMScorer('idwiki.arpa', alpha, beta)
decoder = ctcdecode.BeamSearchDecoder(
    vocabulary,
    num_workers=6,
    beam_width=64,
    scorers=[word_lm_scorer],
    cutoff_prob=np.log(0.000001),
    cutoff_top_n=40
)

found 1gram
found 2gram


In [15]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

test_dataset = load_dataset("common_voice", "id", split="test")
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("indonesian-nlp/wav2vec2-large-xlsr-indonesian")
model = Wav2Vec2ForCTC.from_pretrained("indonesian-nlp/wav2vec2-large-xlsr-indonesian") 
model.to("cuda")

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”\�]'


# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
  batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

Reusing dataset common_voice (/home/user/.cache/huggingface/datasets/common_voice/id/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0)
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [19]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

  with torch.no_grad():
    logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_strings"] = processor.batch_decode(pred_ids)
  batch["lm_pred_strings"] = decoder.decode_batch(logits.cpu().numpy())
  return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

100%|██████████| 4/4 [00:02<00:00,  1.76it/s]



WER: 14.292374


In [21]:
result.to_csv("result.csv")

993657

In [20]:
print("WER: {:2f}".format(100 * wer.compute(predictions=result["lm_pred_strings"], references=result["sentence"])))

WER: 24.948723


In [ ]:
print("LM Prediction: ", text)
print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference: ", test_dataset[:5]["sentence"])